In [1]:
# my packages
from evaluation_table import EvalTable
from figure_generator import EvalPlot
from model import CustomBiLSTM
from tuning_tools import tuning_game, tune_model 
from data_preprocess import data_prepare, data_split
from final_eval import general_viz, regime_eval, signature_eval, eval_drought

# basic packages
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import math
import joblib

# system packages
from datetime import datetime, date, timedelta
import pickle
import warnings
warnings.filterwarnings("ignore")
import platform
import time
from tqdm import tqdm
import os

# hydrological packages
import hydroeval as he
from hydrotools.nwm_client import utils # I had to pip install this

# data analysis packages
from scipy import optimize
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import optuna

# deep learning packages
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split

# Identify the path
home = os.getcwd()
parent_path = os.path.dirname(home)
input_path = f'{parent_path}/02.input/'
output_path = f'{parent_path}/03.output/'
main_path = home

/home/jovyan/envs/devcon_2025/lib/python3.12/site-packages/pyproj/__init__.py:89: UserWarning: pyproj unable to set database path.
  _pyproj_global_context_initialize()


In [2]:
# Load the train and test dataset
data_train = pd.read_pickle(f"{output_path}train_dataset.pkl")
data_test = pd.read_pickle(f"{output_path}test_dataset.pkl")

station_list = list(data_test.station_id.unique())

length_lookback = 2
x_train_scaled, y_train_scaled, x_test_scaled, y_test_scaled, scaler_x, scaler_y, y_train, x_test, y_test = data_prepare(data_train, data_test, length_lookback=length_lookback)


## 4. Model Development 
#### 4.1. Defining the Model
- As mentioned, we will use a Bidirectional LSTM model which has a simple two layer architecture and Pytorch library. 
- The first layer in our model is a bidirecional LSTM layer which is similar to normal LSTM and the only difference is that you have to turn 'bidirectional' variable to 'True' in the layer variables. 
- The second layer is fully connected layer which will get the ouptuts of LSTM layer, so we should multiple the neurans number (hidden_size variable) by two. 

* **`batch_size`** Batch size determines how many samples are processed before the model’s weights are updated. Smaller batches offer more frequent updates, while larger batches can provide more stable gradient estimates.

* **`learning_rate`** The learning rate in neural networks controls how much the model’s weights are updated during training. A small learning rate leads to slower but more stable convergence, while a large one can speed up training but may cause the model to overshoot optimal solutions or diverge. It’s a critical hyperparameter that significantly affects training performance and outcomes.

* **`hidden_size`** Hidden size refers to the number of units (neurons) in each hidden layer of the network, controlling the model’s capacity to learn patterns. In LSTMs, it defines the dimensionality of the hidden state and cell state.

* **`num_layers`** Number of layers indicates how many stacked layers of LSTM cells the model has. More layers allow the network to learn more complex representations, but also increase the risk of overfitting and training instability.

#### 4.2. Tuning the Hyperparameters
- We have several hyperparameters for our LSTM model, which we have to tune so that we can have the best possible results. 
- The tunning process can be done manually or by using optimization algorithms, in this tutorial we will use the values that we have identified to work best. 

In [3]:

selected_station = station_list[0]
epochs = 10 # We don't change it.
input_size = x_train_scaled[selected_station].shape[2]
# Move the model and data to GPU. 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Feed the data to DataLoader and TensorDataset Functions
x_train_tensor = torch.Tensor(x_train_scaled[selected_station].astype(float))
y_train_tensor = torch.Tensor(y_train_scaled[selected_station].astype(float))
train_dataset = TensorDataset(x_train_tensor, y_train_tensor)

# Define the initial parameters for the LSTM model.
params = {
    'batch_size': 50,
    'learning_rate': 1e-4,
    'hidden_size': 300,
    'num_layers': 1,
}

tuning_game(input_size, device, train_dataset, epochs, params, selected_station)

Initial score: 0.2021058531681804 with params: {'batch_size': 50, 'learning_rate': 0.0001, 'hidden_size': 300, 'num_layers': 1}


Do you want to change any variable? (y/n):  y
Which variable number? (batch_size(1)/learning_rate(2)/hidden_size(3)/num_layers(4)): 4
Enter the new value for num_layers (previous value 1):  2


Previous Mean Score: 0.202
New Mean Score: 0.183 


Do you want to keep the new variable?(y/n):  y
Do you want to change any variable? (y/n):  n


Finished tuning.
Final parameters: {'batch_size': 50, 'learning_rate': 0.0001, 'hidden_size': 300, 'num_layers': 2}.


{'batch_size': 50,
 'learning_rate': 0.0001,
 'hidden_size': 300,
 'num_layers': 2}

In [7]:

# Initialize empty DataFrames to store evaluation results if not already defined.
EvalDF_all_rf = pd.DataFrame()
SupplyEvalDF_all_rf = pd.DataFrame()
df_eval_rf = pd.DataFrame()
df_result_data= {}



bilstm_model = CustomBiLSTM(input_size, params['hidden_size'], params['num_layers'], 1, device, embedding=False, station_list=station_list)

x_test_tensor = torch.Tensor(x_test_scaled[selected_station].astype(float))
y_test_tensor = torch.Tensor(y_test_scaled[selected_station].astype(float))
test_dataset = TensorDataset(x_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=test_dataset.tensors[0].shape[0], shuffle=False)
yhat_test_scaled, val_loss = bilstm_model.evaluate_model(test_loader)

# Inverse transform the scaled predictions to their original scale.
yhat_test = scaler_y.inverse_transform(yhat_test_scaled.reshape(-1, 1))

# Assuming EvalTable is a predefined function that compares predictions to actuals and returns evaluation DataFrames.
EvalDF_all_rf_temp, SupplyEvalDF_all_rf_temp, df_eval_rf_temp = EvalTable(yhat_test.reshape(-1), data_test[data_test.station_id == selected_station][2:], 'lstm')

df_result_data[selected_station] = data_test[data_test.station_id == selected_station][2:].copy()

df_result_data[selected_station]['lstm_flow'] = yhat_test

# Append the results from each station to the respective DataFrame.
EvalDF_all_rf = pd.concat([EvalDF_all_rf, EvalDF_all_rf_temp], ignore_index=True)
SupplyEvalDF_all_rf = pd.concat([SupplyEvalDF_all_rf, SupplyEvalDF_all_rf_temp], ignore_index=True)
df_eval_rf = pd.concat([df_eval_rf, df_eval_rf_temp], ignore_index=True)

print("Model Performance for Daily cfs")
display(EvalDF_all_rf)   
print("Model Performance for Daily Accumulated Supply (Acre-Feet)")
display(SupplyEvalDF_all_rf)

ValueError: Length of values (6021) does not match length of index (6029)

#### 4.3. Automatic Tuning

In [5]:
length_lookback = 10
x_train_scaled, y_train_scaled, x_test_scaled, y_test_scaled, scaler_x, scaler_y, y_train, x_test, y_test = data_prepare(data_train, data_test, length_lookback=length_lookback)
selected_station = station_list[0]
epochs = 5 
input_size = x_train_scaled[selected_station].shape[2]

# Move the model and data to the GPU if available. 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Compute lengths for 80/20 split
x_train_tensor = torch.Tensor(x_train_scaled[selected_station].astype(float))
y_train_tensor = torch.Tensor(y_train_scaled[selected_station].astype(float))
train_dataset = TensorDataset(x_train_tensor, y_train_tensor)

train_len = int(len(train_dataset) * 0.8)
val_len = len(train_dataset) - train_len

# Split dataset
train_dataset, val_dataset = random_split(train_dataset, [train_len, val_len])

# Create DataLoaders
train_loader = {selected_station: DataLoader(train_dataset, batch_size=50, shuffle=True)}
val_loader = {selected_station: DataLoader(val_dataset, batch_size=50, shuffle=False)}

def objective(trial):
    # Suggest hyperparameters
    hidden_size = trial.suggest_int("hidden_size", 32, 128)
    num_layers = trial.suggest_int("num_layers", 1, 3)
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-2, log=False)

    # Create the Model
    bilstm_model = CustomBiLSTM(input_size, hidden_size, num_layers, 1, device, embedding=False, station_list=station_list[0:1])
    
    # Create the Optimizer
    bilstm_optimizer = optim.Adam(bilstm_model.parameters(), lr=learning_rate, weight_decay=0)
    
    # Run the training function
    model_parameters = bilstm_model.train_model(train_loader, epochs, bilstm_optimizer, early_stopping_patience=0, val_loader=None, tune='True')
        # print('hi')
    outputs, val_loss = bilstm_model.evaluate_model(val_loader[selected_station])
    return val_loss  # Minimize validation loss
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)

print("Best hyperparameters:", study.best_params)


[I 2025-05-21 15:47:36,507] A new study created in memory with name: no-name-7301b78f-7cb6-46dc-b9f4-39c76a05b568
[I 2025-05-21 15:47:38,812] Trial 0 finished with value: 0.13322460696967006 and parameters: {'hidden_size': 128, 'num_layers': 1, 'learning_rate': 0.003394573297864832}. Best is trial 0 with value: 0.13322460696967006.
[I 2025-05-21 15:47:41,630] Trial 1 finished with value: 0.12717216416327595 and parameters: {'hidden_size': 42, 'num_layers': 2, 'learning_rate': 0.00843941366703184}. Best is trial 1 with value: 0.12717216416327595.
[I 2025-05-21 15:47:46,311] Trial 2 finished with value: 0.14027943476563687 and parameters: {'hidden_size': 126, 'num_layers': 2, 'learning_rate': 0.0047380284580664424}. Best is trial 1 with value: 0.12717216416327595.
[I 2025-05-21 15:47:52,323] Trial 3 finished with value: 0.13756137659486523 and parameters: {'hidden_size': 103, 'num_layers': 3, 'learning_rate': 0.0035406498999942366}. Best is trial 1 with value: 0.12717216416327595.
[I 202

Best hyperparameters: {'hidden_size': 42, 'num_layers': 2, 'learning_rate': 0.00843941366703184}


In [9]:
params = study.best_params
params['batch_size'] = 50

joblib.dump(params, f'{output_path}best_hyperparameters_lstm.pkl')

['/home/jovyan/mydrive/devcon_2025/hydromachine-tutorials/neural_nets/lstm/03.output/best_hyperparameters_lstm.pkl']

[**LETS GO TO THE NEXT PART**](./03.tutorial_post_processing_lstm_evaluation.ipynb)